In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

def sine_gaussian_waveform(t, A, t0, tau, f0, phi0):
    '''
    Parameters:
    t : Array of time values
    A : Amplitude of the waveform
    t0 : Central time of the Gaussian envelope
    tau : Width of the Gaussian envelope
    f0 : Frequency of the sine wave
    phi0 : Phase of the sine wave

    Returns an array of values of the generated sine_gaussian waveform
    '''
    gaussian_envelope = A * np.exp(-((t - t0)**2) / (2 * tau**2))
    sine_wave = np.cos(2 * np.pi * f0 * t + phi0)
    return gaussian_envelope * sine_wave

def generate_waveform_data(num_samples, num_points, A_base, A_variation, t0_base, t0_variation, tau, f0_base, f0_variation, phi0, noise_level_base, noise_variation, time_length_base, time_length_variation, signal_present=True):
    '''
    Function generates the given sample amount of waveforms with a corresponding label indicating
    a signal with noise or if it's just pure noise.
    '''
    data = [] 
    labels = []
    
    for _ in range(num_samples):
        # Generate waveform with slight variations from base values
        A = A_base + np.random.uniform(-A_variation, A_variation)
        t0 = t0_base + np.random.uniform(-t0_variation, t0_variation)
        f0 = f0_base + np.random.uniform(-f0_variation, f0_variation)
        noise_level = abs(noise_level_base + np.random.uniform(-noise_variation, noise_variation))
        time_length = time_length_base + np.random.uniform(-time_length_variation, time_length_variation)
        
        t = np.linspace(0, time_length, num_points)

        # Generate signal with noise or pure noise
        if signal_present:
            waveform = sine_gaussian_waveform(t, A, t0, tau, f0, phi0)
            noise = np.random.uniform(-noise_level, noise_level, t.shape)
            signal_with_noise = waveform + noise
        else:
            signal_with_noise = np.random.uniform(-noise_level, noise_level, t.shape)

        # Sample points
        sample_indices = np.linspace(0, num_points - 1, 4096, dtype=int)
        sampled_amplitudes = signal_with_noise[sample_indices]
        
        features = sampled_amplitudes
        
        data.append(features)
        
        # Generate labels
        if signal_present:
            label = np.zeros_like(sampled_amplitudes)
            signal_start = np.argmin(np.abs(t - t0 + 3 * tau))  # Approx start point of signal
            signal_end = np.argmin(np.abs(t - t0 - 3 * tau))    # Approx end point of signal
            label[signal_start:signal_end] = 1
            labels.append(label)
        else:
            labels.append(np.zeros_like(sampled_amplitudes))
    
    return np.array(data), np.array(labels)

In [8]:
# Parameters for waveform generation
num_samples_per_class = 10 # Number of samples for each class (signal/no signal)
num_points = 4096  # Number of time points that are extracted from each time series sample

# Define base parameters for waveform generation
A_base = 0.5e-22  # Base amplitude
t0_base = 0.5  # Base central time (seconds)
tau = 0.05  # Width of the Gaussian envelope (seconds)
f0_base = 100  # Base frequency of the sine wave (Hz)
phi0 = 0.0  # Phase of the sine wave (radians)

# Variations in parameters for waveform generation
A_variation = 0e-22  # Variation range for amplitude
t0_variation = 0.3  # Variation range for central time
f0_variation = 10  # Variation range for frequency (Hz)
noise_level_base = 3e-22  # Base noise level
noise_variation = 0  # Variation range for noise level
time_length_base = 1.0  # Base time length (seconds)
time_length_variation = 0.3  # Variation range for time length (seconds)

# Generate data with and without signal
data_with_signal, labels_with_signal = generate_waveform_data(
    num_samples_per_class, num_points, A_base, A_variation, t0_base, t0_variation, 
    tau, f0_base, f0_variation, phi0, noise_level_base, noise_variation, 
    time_length_base, time_length_variation, signal_present=True
)
data_without_signal, labels_without_signal = generate_waveform_data(
    num_samples_per_class, num_points, A_base, A_variation, t0_base, t0_variation, 
    tau, f0_base, f0_variation, phi0, noise_level_base, noise_variation, 
    time_length_base, time_length_variation, signal_present=False
)

# Combine data and labels
data = np.concatenate([data_with_signal, data_without_signal])
labels = np.concatenate([labels_with_signal, labels_without_signal])

# Shuffle the data and labels together
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# Scale each sample by the base amplitude used for the Gaussian sine wave
data = data / A_base #change normalisation here 

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=42)

# Transpose the training and test data
X_train_transposed = X_train.T
X_test_transposed = X_test.T

#print(y_train.shape)  # (300, 4096)

# Create labels array where each row is filled with the respective label value for training data
y_train_expanded = y_train

# Save training data to CSV file
train_df = pd.DataFrame(X_train_transposed)
train_df.to_csv('train.csv', index=False, )

# Save test data to CSV file
test_df = pd.DataFrame(X_test_transposed)
test_df.to_csv('test.csv', index=False)

# Save training labels to CSV file
labels_train_df = pd.DataFrame(y_train.T)
print(labels_train_df.shape)  # Should print (4096, 300)
labels_train_df.to_csv('labels.csv', index=False)

# Save test labels to CSV file
labels_test_df = pd.DataFrame(y_test.T)
print(labels_test_df.shape)  # Should print (4096, 300)
labels_test_df.T.to_csv('test_labels.csv', index=False)

# Verify the shapes of saved data
#print(f'Training data shape: {X_train_transposed.shape}')
#print(f'Test data shape: {X_test_transposed.shape}')
#print(f'Training labels shape: {test_df.shape}')

(4096, 10)
(4096, 10)
